# Predicting churn with the SPSS random tree algorithm

This shows you how to create a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark version 2.0. You'll learn how to create an SPSS random tree model by using the IBM SPSS Machine Learning API, and how to view the model with IBM SPSS Model Viewer.

Because it consists of multiple classification and regression trees (CART), you can use random tree algorithms to generate accurate predictive models and solve complex classification and regression problems. Each tree develops from a bootstrap sample that is produced by resampling the original data points with replacement data. During the resampling phase, the best split variable is selected for each node from a specified smaller number of variables that are drawn randomly from the full set of variables. Each tree grows without pruning and then, during the scoring phase, the random tree algorithm aggregates tree scores by majority voting (for classification) or average (for regression).

In this notebook, you'll create a model with telecommunications data to predict when its customers will leave for a competitor, so that you can take some action to retain the customer.
    
To get the most out of this notebook, you should have some familiarity with the Scala programming language.

This notebooks runs on Scala and Spark. Some familiarity with Scala is recommended.

## Contents 
This notebook contains the following main sections:

1. [Load the Telco Churn data to the cloud data repository.](#overview)
1. [Prepare the data.](#prepare)
1. [Configure the RandomTrees model.](#configure) 
1. [View the model.](#view)
1. [Summary and next steps.](#next)    

<a id="overview"></a>
## 1. Load the Telco Churn data to the cloud data repository
Telco Churn is a hypothetical data file that concerns a telecommunications company's efforts to reduce turnover in its customer base. Each case corresponds to a separate customer and it records various demographic and service usage information. Before you can work with the data, you must use the URL to get the telco.csv and telco_Feb.csv files from the GitHub repository.


In [1]:
val link_telco = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco) #> new File("telco.csv") !!

val link_telco_Feb = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco_Feb.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco_Feb) #> new File("telco_Feb.csv") !!

<a id="prepare"></a>
## 2. Prepare the data

After uploading the CSV files that contain the data, you must create a SQLContext, put the data from the telco.scv file into a Spark DataFrame, and show the first row in the DataFrame.

In [2]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfTelco = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco.csv")

dfTelco.show(1)

+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------+-------+-------+----------------+-------+----------------+-------+-----+
|region|tenure|age|marital|address|income| ed|employ|retire|gender|reside|tollfree|equip|callcard|wireless|longmon|tollmon|equipmon|cardmon|wiremon|longten|tollten|equipten|cardten|wireten|multline|voice|pager|internet|callid|callwait|forward|confer|ebill|         loglong|logtoll|logequi|         logcard|logwire|           lninc|custcat|churn|
+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------+-------+-------+----------------+--

Review the data. Print the schema of the DataFrame to look at what kind of data you have.

In [3]:
dfTelco.printSchema

root
 |-- region: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital: integer (nullable = true)
 |-- address: integer (nullable = true)
 |-- income: integer (nullable = true)
 |-- ed: integer (nullable = true)
 |-- employ: integer (nullable = true)
 |-- retire: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- reside: integer (nullable = true)
 |-- tollfree: integer (nullable = true)
 |-- equip: integer (nullable = true)
 |-- callcard: integer (nullable = true)
 |-- wireless: integer (nullable = true)
 |-- longmon: double (nullable = true)
 |-- tollmon: double (nullable = true)
 |-- equipmon: double (nullable = true)
 |-- cardmon: double (nullable = true)
 |-- wiremon: double (nullable = true)
 |-- longten: double (nullable = true)
 |-- tollten: double (nullable = true)
 |-- equipten: double (nullable = true)
 |-- cardten: double (nullable = true)
 |-- wireten: double (nullable = true)
 |-- multline: int

Create a DataFrame for the telco_Feb.csv data. You'll use this year's data to build the model, and use the February data for accuracy value.

In [4]:
val dfTelcoFeb = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco_Feb.csv")

<a id="configure"></a>
## 3. Configure the RandomTrees model

By running this portion of the code, you create the random trees estimator, import the libraries, and set the ordinal and nominal variables. Because no inputFieldList value is set, all fields except the target, frequency, and analysis weight fields are treated as input fields. To make the random tree model build faster, set the number of trees to 10 instead of the default value, which is 100. Finally, you must specify the churn target field. 

In [5]:
import com.ibm.spss.ml.classificationandregression.ensemble.RandomTrees
import com.ibm.spss.ml.utils.DataFrameImplicits.DataFrameEnrichImplicitsClass

val ordinal = Array("ed")
val nominal = Array("region",
     "marital",
     "retire",
     "gender",
     "tollfree",
     "equip",
     "callcard",
     "wireless","multline",
     "voice","pager","internet","callid","callwait","forward","confer",
     "ebill",
     "custcat",
     "churn"
   )
val srf = RandomTrees().setTargetField("churn").setNumTrees(10)
val srfModel = srf.fit(dfTelco.setNominalMeasure(nominal,true).setOrdinalMeasure(ordinal,true))

Do the prediction and get your results.

In [6]:
val predResult = srfModel.transform(dfTelcoFeb)
val predResultNew = predResult.withColumn("prediction", predResult("prediction").cast("double")).
    withColumn("churn", predResult("churn").cast("double"))

To get the accuracy result, use the Apache Spark **MulticlassClassificationEvaluator** function. Notice that the accuracy is above 90%.

In [7]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator().setLabelCol("churn").setMetricName("accuracy")
val acc_result = evaluator.evaluate(predResultNew)
println(s"acc_result:$acc_result")

acc_result:0.942


<a id="view"></a>
## 4. View the model

View the model with the SPSS Model Viewer. The visualization for the generalized linear model includes a confusion matrix, a table with top decision rules, and a table and chart of predictor importance.

### 4.1 Generate a project token

Before you can run the model viewer, you need to generate a project token

1. In the **My Projects** banner, click the **More** icon and then click **Insert project token**. The project token is inserted into the first cell of the notebook, before the title.
2. Copy the text, which appears at the beginning of the notebook, into the following cell and run it.

### 4.2 Start the model viewer

Run the code in the following cell to start SPSS Model Viewer, where you can see a visualization and see model statistics and other characteristics.

In [9]:
import com.ibm.spss.scala.ModelViewer
kernel.magics.html(ModelViewer.toHTML(pc,srfModel))

<!DOCTYPE HTML>
 
 
 
 Model Visualization 
 

 <!-- Global Model Visualization includes -->
 
 
 
 
 
 
 

 
 
 
 


 
 Random Trees
 
 
 The Random Trees algorithm is a sophisticated modern approach to supervised learning for categorical or continuous targets. The algorithm uses groups of classification or regression trees and randomness to make predictions that are particularly robust when applied to new observations. The IBM SPSS Spark Machine Learning Library implementation features a table of top decision rules for classification models without imbalance handling and measures of relative predictor importance for all models. For more information, visit the Random trees page on the Data Science Experience web site. 
 
 
 
 
 Model Information 
 Records Summary 
 Predictor Importance 
 Top Decision Rules 
 Confusion Matrix 
 

 
   
   
 
 
 
 
 
 
 
 Missing Project Token - The toHTML method now
 requires a ProjectContext parameter as the first parameter. For more information, refer to the project token documentation page. This API will cease to work without a ProjectContext
 parameter in the near future.
 
 

 
 Shows model settings and other input, and provides summary measures to help you assess the model. ">
 

 
 
 
 Target Field 
 churn 
 
 Model Building Method 
 Random Trees Classification 
 
 Number of Predictors Input 
 36 
 
 Model Accuracy 
 0.695 
 
 Misclassification Rate 
 0.305 
 

 
 

 
 Shows the number and percentage of records included and excluded from the analysis. ">
 


 
 
 
 Records 
 Number 
 Percent 
 
 
 Included 
 1,000 
 100.00 
 
 Excluded 
 0 
 0.00 
 
 Total 
 1,000 
 100.00 
 

 
 

 
 Shows the relative importance of each predictor in estimating the model. ">

<!-- ***************** Notebook Calling = TRUE; width=540 ; formatOptions.getImageSizeX() = 680; NOT Defined *************** -->
 
 
 
	
 

 

 
 
 
 
 
 tenure 
 Importance 
 .098 
 
 
 
 
 
 Mean 35.526 
 Standard Deviation 21.349 
 Minimum 1.000 
 Maximum 72.000 
 N 1,000 
 
 
 
 
 
 
 
 age 
 Importance 
 .093 
 
 
 
 
 
 Mean 41.684 
 Standard Deviation 12.553 
 Minimum 18.000 
 Maximum 77.000 
 N 1,000 
 
 
 
 
 
 
 
 address 
 Importance 
 .079 
 
 
 
 
 
 Mean 11.551 
 Standard Deviation 10.082 
 Minimum .000 
 Maximum 55.000 
 N 1,000 
 
 
 
 
 
 
 
 employ 
 Importance 
 .069 
 
 
 
 
 
 Mean 10.987 
 Standard Deviation 10.077 
 Minimum .000 
 Maximum 47.000 
 N 1,000 
 
 
 
 
 
 
 
 region 
 Importance 
 .061 
 
 
 
 
 
 Value Count Percent 
 
 
 2 334 33.4 

 3 344 34.4 

 1 322 32.2 
 
 
 
 
 
 
 
 longmon 
 Importance 
 .057 
 
 
 
 
 
 Mean 11.723 
 Standard Deviation 10.358 
 Minimum .900 
 Maximum 99.950 
 N 1,000 
 
 
 
 
 
 
 
 income 
 Importance 
 .053 
 
 
 
 
 
 Mean 77.535 
 Standard Deviation 106.991 
 Minimum 9.000 
 Maximum 1,668.000 
 N 1,000 
 
 
 
 
 
 
 
 ed 
 Importance 
 .044 
 
 
 
 
 
 Value Count Percent 
 
 
 1 204 20.4 

 2 287 28.7 

 3 209 20.9 

 4 234 23.4 

 5 66 6.6 
 
 
 
 
 
 
 
 longten 
 Importance 
 .043 
 
 
 
 
 
 Mean 574.050 
 Standard Deviation 789.579 
 Minimum .900 
 Maximum 7,257.600 
 N 1,000 
 
 
 
 
 
 
 
 tollmon 
 Importance 
 .035 
 
 
 
 
 
 Mean 13.274 
 Standard Deviation 16.894 
 Minimum .000 
 Maximum 173.000 
 N 1,000 
 
 
 
 
 
 
 
 lninc 
 Importance 
 .034 
 
 
 
 
 
 Mean 3.957 
 Standard Deviation .803 
 Minimum 2.197 
 Maximum 7.419 
 N 1,000 
 
 
 
 
 
 
 
 cardmon 
 Importance 
 .033 
 
 
 
 
 
 Mean 13.781 
 Standard Deviation 14.077 
 Minimum .000 
 Maximum 109.250 
 N 1,000 
 
 
 
 
 
 
 
 reside 
 Importance 
 .028 
 
 
 
 
 
 Mean 2.331 
 Standard Deviation 1.435 
 Minimum 1.000 
 Maximum 8.000 
 N 1,000 
 
 
 
 
 
 
 
 loglong 
 Importance 
 .028 
 
 
 
 
 
 Mean 2.182 
 Standard Deviation .734 
 Minimum -.105 
 Maximum 4.605 
 N 1,000 
 
 
 
 
 
 
 
 cardten 
 Importance 
 .021 
 
 
 
 
 
 Mean 605.774 
 Standard Deviation 829.711 
 Minimum .000 
 Maximum 7,515.000 
 N 1,000 
 
 
 
 
 
 
 
 equipmon 
 Importance 
 .020 
 
 
 
 
 
 Mean 14.220 
 Sta

### 4.3. Export the XML files (PMML, StatXML) for other detail statistics
By exporting your results to different formats, such as Predictive Model Markup Language (PMML) or statXML format you can share your statistical analyses outside of IBM Data Science Experience.

In [10]:
import java.io.{File, PrintWriter}

srfModel.toPMML("randomTrees_pmml.xml")
val statXML = srfModel.statXML()
new PrintWriter("StatXML.xml") {
      write(statXML)
      close
}

$anon$1@59740911

<a id="next"></a>
# Summary and next steps
You have created a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark. Now you can create a different model to compare model evaluations, such as the test of model effects, residuals, and so on. See [SPSS documentation](https://apsportal.ibm.com/docs/content/kc_gen/integrations-gen2.html).

## Authors

Wang Zhiyuan and Yu Wenpei are SPSS Algorithm Engineers at IBM.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.